In [1]:
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
from keras_retinanet import models
import matplotlib.pyplot as plt
from tensorflow import keras
from pylab import rcParams
from matplotlib import rc
import tensorflow as tf
import seaborn as sns
from PIL import Image
import pandas as pd
import numpy as np
import urllib
import time
import os
import csv
import cv2


%config InlineBackend.figure_format='retina'
%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 22, 10
register_matplotlib_converters()


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

2022-09-12 12:57:51.425704: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-12 12:57:51.619820: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-12 12:57:52.269547: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-09-12 12:57:52.269648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer

In [2]:
df = pd.read_csv('/home/jupyter/notebook-javi/object_detection/tensorflow/labels_dataset2.csv')

df['image_name'] = '/home/jupyter/main_dataset/train/data/' + df.img_filename
df = df.rename(columns={'ann_bbox_xmin':'x_min', 'ann_bbox_ymin':'y_min', 'ann_bbox_xmax': 'x_max', 'ann_bbox_ymax': 'y_max', 'cat_name': 'class_name'})
df = df[['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'class_name']]

In [3]:
df.x_min = df.x_min.apply(int)
df.y_min = df.y_min.apply(int)
df.x_max = df.x_max.apply(int)
df.y_max = df.y_max.apply(int)

In [4]:
train_df, test_df = train_test_split(
  df, 
  test_size=0.2, 
  random_state=RANDOM_SEED
)

In [5]:
ANNOTATIONS_FILE = 'annotations_dataset2.csv'
CLASSES_FILE = 'classes_dataset2.csv'

In [6]:
train_df.to_csv(ANNOTATIONS_FILE, index=False, header=None)

In [7]:
classes = set(df['class_name'])

with open(CLASSES_FILE, 'w') as f:
    for i, line in enumerate(sorted(classes)):
        f.write('{},{}\n'.format(line,i))

In [10]:
PRETRAINED_MODEL = '/home/jupyter/keras-retinanet/snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to /home/jupyter/keras-retinanet/snapshots/_pretrained_model.h5


- El entrenamiento crea una carpeta llamada **snapshots**, donde muestra los pesos entrenados por el modelo en cada época. El que nos interesa es el de la última época.

In [11]:
start_time = time.time()
! /home/jupyter/keras-retinanet/keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 3 --steps 64 --epochs 4 csv annotations_dataset2.csv classes_dataset2.csv
print("--- %s seconds ---" % (time.time() - start_time))

2022-09-12 13:05:40.729152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-12 13:05:40.916938: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-12 13:05:41.734356: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-09-12 13:05:41.734449: W tensorflow/stream_executor/platform/default/dso_l